In [ ]:
import os
import json
import pandas as pd
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
import decimal
import logging

In [ ]:
container_name = "#####"
storage_name = "#####"
mount_dir = "/data"
key = "#####"

In [ ]:


output_container_path = "wasbs://%s@%s.blob.core.windows.net" % (output_container_name, storage_name)

dbutils.fs.mount(
  source = output_container_path,
  mount_point = mount_dir,
  extra_configs = {"fs.azure.sas.%s.%s.blob.core.windows.net" % (container_name, storage_name): key })

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2211900603944736> in <module> 
 5 
 6 
 ----> 7 dbutils.fs.mount(
 8 source = "wasbs://%s@%s.blob.core.windows.net" % ( container_name , storage_name ) , 
 9 mount_point = mount_dir , 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 379 exc . __context__ = None 
 380 exc . __cause__ = None 
 --> 381 raise exc
 382 
 383 return f_with_exception_handling

 ExecutionError : An error occurred while calling o316.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: Mount point '/data' must be within '/mnt'; nested exception is: 
	java.lang.IllegalArgumentException: Mount point '/data' must be within '/mnt'
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:739)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: Mount point '/data' must be within '/mnt'
	at com.databricks.backend.daemon.data.common.MountConf.ensureMountPointValid(MountConf.scala:80)
	at com.databricks.backend.daemon.data.common.MountConf.ensureMountPointValid$(MountConf.scala:59)
	at com.databricks.backend.daemon.data.server.DataDaemonConf.ensureMountPointValid(DataDaemonConf.scala:19)
	at com.databricks.backend.daemon.data.server.util.MountEntry$.fromDbfsMountPoint(MountEntry.scala:248)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.validateMountRequest(MountHandler.scala:56)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:95)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:300)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:259)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:90)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:117)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:117)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:75)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:346)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:440)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:460)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:213)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.loggi

In [ ]:
# A set of functions

def parse_csv(line):
    """
   This function will parse ".txt" file from blob storage.

   since we are working with comma-separated values file so we want to return event object

   :param line: A line of ".txt" file in CSV format.
   :return: Common event object.
   """
    try:
        record_type_pos = 2 # filter for "Q" or "T" value
        record = line.split(",")
        # Filter by record_type (T = Trade or Q = Quote):
        if record[record_type_pos] == "T":
            # Create event object by using values from record object and performing data type conversion
            event = [record[0], # trade_dt
                     record[1], # file_tm
                     record[2], # event_type
                     record[3], # symbol
                     record[4], # event_tm
                     int(record[5]), # event_seq_nb
                     record[6],   # exchange
                     decimal.Decimal(record[7]), # bid_pr
                     int(record[8]), # bid_size
                     None, None,
                     "T"]
            return event
        elif record[record_type_pos] == "Q":
            event = [record[0], 
                     record[1], 
                     record[2], 
                     record[3], 
                     record[4], 
                     int(record[5]), 
                     record[6],
                     decimal.Decimal(record[7]), 
                     int(record[8]), 
                     decimal.Decimal(record[9]), # ask_pr
                     int(record[10]),            # ask_size
                     "Q"]
            return event
    except Exception as e:
        # Return exception as "Bad record" and convert values to None preceding record_type == "B"
        event = [None, None, None, None, None, None, None, None, None, None, None, "B"]
        logging.error("Bad record", e)
        # print(f"Bad record: {e}")
        return event
    
def parse_json(line):
    """
    This function will parse through each line in the JSON formatted ".txt" file stored from blob storage.

    :param line: Each line of ".txt" file in JSON format.
    :return: common_event() object
    """
    try:
        record = json.loads(line)
        record_type = record["event_type"]
        # Parse records for each type and convert data type as necessary
        # Filter by record_type (T = Trade or Q = Quote):
        if record_type == "T":
            # Create event object based and
            event = [record["trade_dt"], 
                     record["file_tm"], 
                     record["event_type"], 
                     record["symbol"],
                     record["event_tm"], 
                     int(record["event_seq_nb"]), 
                     record["exchange"],
                     decimal.Decimal(record["bid_pr"]), 
                     int(record["bid_size"]), 
                     None, None, # Try place None values
                     "T"]
            return event
        elif record_type == "Q":
            event = [record["trade_dt"], 
                     record["file_tm"], 
                     record["event_type"], 
                     record["symbol"],
                     record["event_tm"], 
                     int(record["event_seq_nb"]), 
                     record["exchange"],
                     decimal.Decimal(record["bid_pr"]), 
                     int(record["bid_size"]), 
                     decimal.Decimal(record["ask_pr"]),
                     int(record["ask_size"]), 
                     "Q"]
            return event
    except Exception as e:
        # Return exception as "Bad record" and convert values to None preceding record_type == "B"
        event = [None, None, None, None, None, None, None, None, None, None, None, "B"]
        logging.error("Bad record", e)
        # print(f"Bad record: {e}")
        return event

In [ ]:
# Initialize schema

commonEventSchema = StructType([
            StructField("trade_dt", StringType(), True),
            StructField("file_tm", StringType(), True),
            StructField("record_type", StringType(), True),
            StructField("symbol", StringType(), True),
            StructField("event_tm", StringType(), True),
            StructField("event_seq_nb", IntegerType(), True),
            StructField("trade_pr", StringType(), True),
            StructField("bid_pr", DecimalType(), True),
            StructField("bid_size", IntegerType(), True),
            StructField("ask_pr", DecimalType(), True),
            StructField("ask_size", IntegerType(), True),
            StructField("partition", StringType(), True)

])

In [ ]:
# csv path
csv_dir_1 = "/data/csv/2020-08-05/NYSE/part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt"
csv_dir_2 = "/data/csv/2020-08-06/NYSE/part-00000-214fff0a-f408-466c-bb15-095cd8b648dc-c000.txt"

json_dir_1 = "/data/json/2020-08-05/NASDAQ/part-00000-c6c48831-3d45-4887-ba5f-82060885fc6c-c000.txt"
json_dir_2 = "/data/json/2020-08-06/NASDAQ/part-00000-092ec1db-39ab-4079-9580-f7c7b516a283-c000.txt"


# Create Spark Session
spark = SparkSession.builder.master('local').appName('app').getOrCreate()

spark.conf.set(
        "fs.azure.account.key.%s.blob.core.windows.net" % (storage_name), 
         key
)

# Initilize spark context
sc = spark.sparkContext

# Raw text files
raw_csv_1 = sc.textFile( "wasbs://%s@%s.blob.core.windows.net%s" %( container_name, storage_name, csv_dir_1))

raw_csv_2 = sc.textFile( "wasbs://%s@%s.blob.core.windows.net%s" %( container_name, storage_name, csv_dir_2))

raw_json_1 = sc.textFile( "wasbs://%s@%s.blob.core.windows.net%s" %( container_name, storage_name, json_dir_1))

raw_json_2 =  sc.textFile( "wasbs://%s@%s.blob.core.windows.net%s" %( container_name, storage_name, json_dir_2))

# Parsed files
parsed_csv1 = raw_csv_1.map(lambda line: parse_csv(line))
parsed_csv2 = raw_csv_2.map(lambda line: parse_csv(line))

parsed_json1= raw_json_1.map(lambda line: parse_json(line))
parsed_json2= raw_json_2.map(lambda line: parse_json(line))

spark_df1 = spark.createDataFrame(parsed_csv1, commonEventSchema)
spark_df2 = spark.createDataFrame(parsed_csv2, commonEventSchema)
spark_df3 = spark.createDataFrame(parsed_json1, commonEventSchema)
spark_df4 = spark.createDataFrame(parsed_json2, commonEventSchema)

spark_df1

Out[6]: DataFrame[trade_dt: string, file_tm: string, record_type: string, symbol: string, event_tm: string, event_seq_nb: int, trade_pr: string, bid_pr: decimal(10,0), bid_size: int, ask_pr: decimal(10,0), ask_size: int, partition: string]

In [ ]:
spark_df1.limit(10).show()

+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+
 trade_dt| file_tm|record_type|symbol| event_tm|event_seq_nb|trade_pr|bid_pr|bid_size|ask_pr|ask_size|partition|
+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 09:50:...| 3| NYSE| 77| 100| 77| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 09:57:...| 4| NYSE| 79| 100| 80| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 10:06:...| 5| NYSE| 78| 100| 78| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 10:11:...| 6| NYSE| 79| 100| 80| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 10:17:...| 7| NYSE| 77| 100| 78| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 10:23:...| 8| NYSE| 78| 100| 79| 100| Q|
2020-08-05|2020-08-05 09:30:...| Q| SYMA|2020-08-05 10:31:...| 9| NYSE| 77| 100| 79| 100| Q|
2020-08-05|2020-08-05 09:30:...| T| SYMA|2020-08-05 10:37:...| 10| NYSE| 79| 912| null| null| T|
+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+

In [ ]:
spark_df4.limit(10).show()

+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+
 trade_dt| file_tm|record_type|symbol| event_tm|event_seq_nb|trade_pr|bid_pr|bid_size|ask_pr|ask_size|partition|
+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 09:38:...| 1| NASDAQ| 78| 100| 80| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 09:46:...| 2| NASDAQ| 77| 100| 77| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 09:52:...| 3| NASDAQ| 79| 100| 79| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 09:58:...| 4| NASDAQ| 76| 100| 77| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 10:07:...| 5| NASDAQ| 77| 100| 79| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 10:15:...| 6| NASDAQ| 79| 100| 81| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 10:22:...| 7| NASDAQ| 78| 100| 79| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 10:29:...| 8| NASDAQ| 76| 100| 77| 100| Q|
2020-08-06|2020-08-06 09:30:...| Q| SYMA|2020-08-06 10:35:...| 9| NASDAQ| 76| 100| 78| 100| Q|
 null| null| null| null| null| null| null| null| null| null| null| B|
+----------+--------------------+-----------+------+--------------------+------------+--------+------+--------+------+--------+---------+

In [ ]:
# Check the distinct partition
spark_df1.select("partition").distinct().show()

+---------+
partition|
+---------+
 Q|
 T|
+---------+

In [ ]:
# check the total rows of spark_df2 data frame
spark_df2.count()

Out[9]: 300

In [ ]:
union_df = spark_df1.union(spark_df2)\
                    .union(spark_df3)\
                    .union(spark_df4)

In [ ]:
# Check the total rows of union data frame
union_df

Out[13]: 1200

In [ ]:
# Check the distinct partition
union_df.select("partition").distinct().show()

+---------+
partition|
+---------+
 Q|
 T|
 B|
+---------+

In [ ]:
# Write The Common Events Into Partitions As Parquet Files To HDFS
union_df.write.partitionBy("partition").mode("overwrite").parquet("output_dir")